In [10]:
from faker import Faker
from faker import Faker
from PIL import Image, ImageDraw, ImageFont
import random
import os

fake = Faker('es_MX')

In [11]:
# Carpeta de destino para guardar las imágenes
CARPETA_DESTINO = r"C:\Users\Adrian\Desktop\INEs Dataset"

# Crear la carpeta si no existe
os.makedirs(CARPETA_DESTINO, exist_ok=True)

In [12]:
# Función para eliminar acentos
def eliminar_acentos(texto):
    reemplazos = {
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U',
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Ü': 'U', 'ü': 'u', 'Ñ': 'N', 'ñ': 'n'
    }
    for acento, sin_acento in reemplazos.items():
        texto = texto.replace(acento, sin_acento)
    return texto

In [13]:
# Función para generar un CURP válido en formato
def generar_curp():
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato básico de CURP mexicano (simplificado)
    curp = ""
    curp += ''.join(random.choices(letras, k=4))  # Primeras 4 letras (apellidos y nombre)
    curp += ''.join(random.choices(numeros, k=6))  # Fecha (AAMMDD)
    curp += random.choice("HM")  # Sexo
    curp += ''.join(random.choices(letras, k=2))  # Estado
    curp += ''.join(random.choices(letras, k=3))  # Consonantes internas
    curp += random.choice(numeros + letras)  # Homoclave
    curp += ''.join(random.choices(numeros, k=1))  # Dígito verificador
    
    return curp

In [14]:
# Función para generar un número de elector válido
def generar_clave_elector():
    letras = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    numeros = "0123456789"
    
    # Formato típico de clave de elector
    clave = ""
    clave += ''.join(random.choices(letras, k=6))  # Primeras letras (apellidos y nombre)
    clave += ''.join(random.choices(numeros, k=8))  # Fecha y código
    clave += ''.join(random.choices(letras + numeros, k=4))  # Homoclave
    
    return clave

In [15]:
# Función para generar datos de persona falsos
def generar_datos_persona():
    # Generar nombre y apellidos separados
    nombre = eliminar_acentos(fake.first_name().upper())
    apellido1 = eliminar_acentos(fake.last_name().upper())
    apellido2 = eliminar_acentos(fake.last_name().upper())
    
    # Determinar género y edad
    genero = random.choice(["H", "M"])
    fecha_nac = fake.date_of_birth(minimum_age=18, maximum_age=90)
    
    # Generar dirección con formato similar a INE (sin acentos)
    calle = eliminar_acentos(fake.street_name().upper())
    numero = fake.building_number()
    colonia = eliminar_acentos(fake.city_suffix().upper() + " " + fake.city_prefix().upper())
    ciudad = eliminar_acentos(fake.city().upper())
    estado = eliminar_acentos(fake.state().upper())
    cp = fake.postcode()
    
    direccion_linea1 = f"{calle} {numero}"
    direccion_linea2 = f"{colonia} {cp}"
    direccion_linea3 = f"{ciudad}, {estado}"
    
    seccion = str(random.randint(1, 9999)).zfill(4)
    
    # Vigencia como rango de años
    año_inicio = random.randint(2020, 2030)
    año_fin = año_inicio + 10
    vigencia = f"{año_inicio}-{año_fin}"
    
    registro = str(random.randint(2000, 2023))
    
    # Fecha de nacimiento en formato DD/MM/YYYY
    fecha_nacimiento = fecha_nac.strftime('%d/%m/%Y')
    
    return {
        "apellido1": apellido1,
        "apellido2": apellido2,
        "nombre": nombre,
        "clave_elector": generar_clave_elector().upper(),
        "curp": generar_curp().upper(),
        "fecha_nacimiento": fecha_nacimiento,
        "direccion1": direccion_linea1,
        "direccion2": direccion_linea2,
        "direccion3": direccion_linea3,
        "sexo": genero,
        "seccion": seccion,
        "vigencia": vigencia,
        "registro": registro
    }

In [ ]:
# Cargar plantilla de INE y generar una falsa
def generar_ine_falsa(persona, output_filename):
    try:
        # Cargar imagen de plantilla
        imagen = Image.open("plantilla_ine.png")
        
        # Convertir a RGB si está en modo RGBA
        if imagen.mode == 'RGBA':
            imagen = imagen.convert('RGB')
            
        draw = ImageDraw.Draw(imagen)
        
        # Configurar fuentes
        try:
            # Todas en negrita si es posible
            fuente_etiqueta = ImageFont.truetype("arial.ttf", size=12)  # Para etiquetas
            fuente_dato = ImageFont.truetype("arial.ttf", size=14)     # Para los datos
            fuente_grande = ImageFont.truetype("arial.ttf", size=16)   # Para nombre
        except IOError:
            fuente_etiqueta = ImageFont.load_default()
            fuente_dato = ImageFont.load_default()
            fuente_grande = ImageFont.load_default()
        
        # Ajustar posiciones para dejar espacio para la foto (mover todo a la derecha)
        offset_x = 270  # Aumentado para dejar más espacio para la foto
        offset_y = 170
        
        # NOMBRE
        draw.text((offset_x, offset_y), "NOMBRE", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 15), persona["apellido1"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 35), persona["apellido2"], font=fuente_grande, fill="black")
        draw.text((offset_x, offset_y + 55), persona["nombre"], font=fuente_grande, fill="black")
        
        # SEXO - reposicionado para no encimarse
        draw.text((500, offset_y), "SEXO", font=fuente_etiqueta, fill="black")
        draw.text((535, offset_y), persona["sexo"], font=fuente_dato, fill="black")
        
        # DOMICILIO - subido
        draw.text((offset_x, offset_y + 90), "DOMICILIO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x, offset_y + 105), persona["direccion1"], font=fuente_dato, fill="black")
        draw.text((offset_x, offset_y + 125), persona["direccion2"], font=fuente_dato, fill="black")
        draw.text((offset_x, offset_y + 145), persona["direccion3"], font=fuente_dato, fill="black")
        
        # CLAVE DE ELECTOR (en la misma línea)
        draw.text((offset_x, offset_y + 180), "CLAVE DE ELECTOR", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 140, offset_y + 180), persona["clave_elector"], font=fuente_dato, fill="black")
        
        # AÑO DE REGISTRO (en la misma línea que CLAVE DE ELECTOR)
        draw.text((450, offset_y + 180), "AÑO DE REGISTRO", font=fuente_etiqueta, fill="black")
        draw.text((500, offset_y + 195), persona["registro"], font=fuente_dato, fill="black")
        
        # CURP (en la misma línea)
        draw.text((offset_x, offset_y + 195), "CURP", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 50, offset_y + 195), persona["curp"], font=fuente_dato, fill="black")
        
        # FECHA DE NACIMIENTO (en la misma línea)
        draw.text((offset_x, offset_y + 210), "FECHA DE NACIMIENTO", font=fuente_etiqueta, fill="black")
        draw.text((offset_x + 160, offset_y + 210), persona["fecha_nacimiento"], font=fuente_dato, fill="black")
        
        # SECCIÓN (en la misma línea)
        draw.text((350, offset_y + 210), "SECCION", font=fuente_etiqueta, fill="black")
        draw.text((400, offset_y + 210), persona["seccion"], font=fuente_dato, fill="black")
        
        # VIGENCIA (en la misma línea)
        draw.text((450, offset_y + 210), "VIGENCIA", font=fuente_etiqueta, fill="black")
        draw.text((510, offset_y + 210), persona["vigencia"], font=fuente_dato, fill="black")
        
        # Ruta completa para guardar el archivo
        ruta_completa = os.path.join(CARPETA_DESTINO, output_filename)
        
        # Guardar la imagen
        imagen.save(ruta_completa)
        print(f"INE falsa generada: {ruta_completa}")
        return True
    except Exception as e:
        print(f"Error al generar INE falsa: {e}")
        return False

In [17]:
# Generar múltiples INEs falsas
def generar_dataset(cantidad):
    generados = 0
    for i in range(cantidad):
        persona = generar_datos_persona()
        if generar_ine_falsa(persona, f"ine_falsa_{i+1:03d}.jpg"):
            generados += 1
    
    print(f"Dataset generado: {generados} de {cantidad} INEs falsas")
    print(f"Las imágenes se guardaron en: {CARPETA_DESTINO}")

In [18]:
# Ejecutar el código
if __name__ == "__main__":
    # Número de INEs a generar
    num_ines = 10
    generar_dataset(num_ines)

INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_001.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_002.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_003.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_004.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_005.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_006.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_007.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_008.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_009.jpg
INE falsa generada: C:\Users\Adrian\Desktop\INEs Dataset\ine_falsa_010.jpg
Dataset generado: 10 de 10 INEs falsas
Las imágenes se guardaron en: C:\Users\Adrian\Desktop\INEs Dataset
